In [1]:
import requests
import pandas as pd
import time
import json
from tqdm import tqdm
import re

def strip_code_block(content: str) -> str:
    """
    如果字符串以 ``` 或 ```json 开头，并以 ``` 结尾，
    则去掉这些 Markdown 代码块标记，只保留中间的 JSON。
    """
    content = content.strip()
    content = re.sub(r"^```(json)?", "", content, flags=re.MULTILINE)
    content = re.sub(r"```$", "", content, flags=re.MULTILINE)
    return content.strip()

# ========== 1. 基础配置 ==========

API_KEY = "sk-"  # 请替换为你的真实 API Key
INPUT_EXCEL = "TEXT.xlsx"       # 输入的 Excel 文件
OUTPUT_EXCEL = "output_file.xlsx"   # 输出结果的 Excel 文件

BASE_URL = "https://api.siliconflow.cn/v1/chat/completions"

VALID_CATEGORIES = {
    "奥运", "国内政治", "国际政治", "军事和防御", "国内秩序", "经济", "劳动关系", "商务活动", "交通",
    "卫生福利", "人口", "教育", "传播", "住房", "环境", "能源", "科技", "社会关系", "灾害", "体育",
    "文化活动", "时尚", "庆典", "人类爱好", "其他"
}
VALID_ATTITUDES = {"positive", "negative", "hostile", "pleasure", "pain"}

# 构建基础的 Payload 配置（注意如果有嵌套字典，copy() 只是浅拷贝）
BASE_PAYLOAD = {
    "model": "Qwen/Qwen2.5-32B-Instruct",  # 请根据需要修改模型名称
    "stream": False,
    "max_tokens": 512,
    "stop": ["null"],
    "temperature": 0.7,
    "top_p": 0.7,
    "top_k": 50,
    "frequency_penalty": 0.5,
    "n": 1,
    "response_format": {"type": "text"},
    "tools": [
        {
            "type": "function",
            "function": {
                "description": "<string>",
                "name": "<string>",
                "parameters": {},
                "strict": False
            }
        }
    ]
}

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

In [2]:
# ========== 2. 定义辅助函数 ==========

def normalize_category(raw_category: str) -> str:
    """
    将模型输出的分类修正为预定义的值，不匹配时返回 '其他'
    """
    raw_category = raw_category.strip()
    for cat in VALID_CATEGORIES:
        if cat in raw_category:
            return cat
    return "其他"

def normalize_attitude(raw_attitude: str) -> str:
    """
    将模型输出的情感修正为预定义的值，不匹配时返回 'neutral'
    """
    raw_attitude = raw_attitude.strip().lower()
    for att in VALID_ATTITUDES:
        if att in raw_attitude:
            return att
    return "neutral"

def get_classification_and_sentiment(title: str, text: str) -> (str, str):
    """
    一次 API 请求，获取文章的分类和情感。
    内部增加了重试机制，确保在获得完整回复后才返回结果。
    """
    prompt = f"""
请阅读以下文章，并给出以下信息（用JSON格式返回）：
1. classification：只能从下列类别中选择：
{list(VALID_CATEGORIES)}
2. sentiment：只能从 [positive, negative, hostile, pleasure, pain] 中选择

请返回 JSON 格式（请注意不要输出任何“```”等多余字符）：
{{
  "classification": "xxx",
  "sentiment": "xxx"
}}

如果无法判断，请将 classification = "其他"；sentiment = "neutral"。

文章标题：{title}
文章内容：{text}
"""
    # 注意这里用浅拷贝即可，messages 字段会覆盖掉原有的同名字段
    payload = BASE_PAYLOAD.copy()
    payload["messages"] = [
        {
            "role": "user",
            "content": prompt
        }
    ]
    
    max_retries = 5
    retry_count = 0
    while retry_count < max_retries:
        try:
            response = requests.post(BASE_URL, json=payload, headers=HEADERS)
            # 如果触发速率限制，通常返回 HTTP 429
            if response.status_code == 429:
                print("触发速率限制 (HTTP 429)，等待60秒后重试...")
                time.sleep(60)
                retry_count += 1
                continue
            elif response.status_code != 200:
                print(f"服务器返回状态码 {response.status_code}，等待10秒后重试...")
                time.sleep(10)
                retry_count += 1
                continue

            # 打印调试信息
            print("===== 调试信息：服务器返回 =====")
            print(response.text)
            print("=============================\n")

            response_json = response.json()
            if "choices" not in response_json or len(response_json["choices"]) == 0:
                print("未获得有效的 choices，返回默认值")
                return "其他", "neutral"

            raw_content = response_json["choices"][0]["message"]["content"]
            raw_content = strip_code_block(raw_content)
            result = json.loads(raw_content)

            raw_cat = result.get("classification", "其他")
            raw_sen = result.get("sentiment", "neutral")
            category = normalize_category(raw_cat)
            sentiment = normalize_attitude(raw_sen)
            return category, sentiment

        except Exception as e:
            print(f"请求或解析时出现异常：{e}，等待10秒后重试...")
            time.sleep(10)
            retry_count += 1

    print("达到最大重试次数，返回默认值")
    return "其他", "neutral"

In [3]:
# ========== 3. 主流程 ==========
df = pd.read_excel(INPUT_EXCEL)
results = []
for idx, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing"):
    art_id = row['id']
    title = row['title']
    text = row['text']

    category, sentiment = get_classification_and_sentiment(title, text)
    results.append({
        'id': art_id,
        'title': title,
        'text': text,
        'classification': category,
        'sentiment': sentiment
    })

        # 为了避免频繁请求导致触发速率限制，每次请求间等待一定时间
    time.sleep(0.5)

Processing:   0%|          | 0/1060 [00:00<?, ?it/s]

===== 调试信息：服务器返回 =====
{"id":"0194dc585546348f0117ea16761d88da","object":"chat.completion","created":1738863564,"model":"Qwen/Qwen2.5-32B-Instruct","choices":[{"index":0,"message":{"role":"assistant","content":"{\n  \"classification\": \"商务活动\",\n  \"sentiment\": \"neutral\"\n}"},"finish_reason":"stop"}],"usage":{"prompt_tokens":2105,"completion_tokens":18,"total_tokens":2123},"system_fingerprint":""}



Processing:   0%|          | 1/1060 [00:01<23:30,  1.33s/it]

===== 调试信息：服务器返回 =====
{"id":"0194dc585a70d151c3a0129bbfd987ec","object":"chat.completion","created":1738863565,"model":"Qwen/Qwen2.5-32B-Instruct","choices":[{"index":0,"message":{"role":"assistant","content":"{\n  \"classification\": \"国际政治\",\n  \"sentiment\": \"neutral\"\n}"},"finish_reason":"stop"}],"usage":{"prompt_tokens":1456,"completion_tokens":18,"total_tokens":1474},"system_fingerprint":""}



Processing:   0%|          | 2/1060 [00:02<24:15,  1.38s/it]

...

Processing: 100%|█████████▉| 1058/1060 [48:59<00:02,  1.32s/it]

===== 调试信息：服务器返回 =====
{"id":"0194dc852e46cc1d51b6b5984cb8e436","object":"chat.completion","created":1738866503,"model":"Qwen/Qwen2.5-32B-Instruct","choices":[{"index":0,"message":{"role":"assistant","content":"{\n  \"classification\": \"交通\",\n  \"sentiment\": \"negative\"\n}"},"finish_reason":"stop"}],"usage":{"prompt_tokens":889,"completion_tokens":17,"total_tokens":906},"system_fingerprint":""}



Processing: 100%|█████████▉| 1059/1060 [49:00<00:01,  1.27s/it]

===== 调试信息：服务器返回 =====
{"id":"0194dc8532da15522cbbad56ba35d488","object":"chat.completion","created":1738866504,"model":"Qwen/Qwen2.5-32B-Instruct","choices":[{"index":0,"message":{"role":"assistant","content":"{\n  \"classification\": \"卫生福利\",\n  \"sentiment\": \"positive\"\n}"},"finish_reason":"stop"}],"usage":{"prompt_tokens":3245,"completion_tokens":18,"total_tokens":3263},"system_fingerprint":""}



Processing: 100%|██████████| 1060/1060 [49:01<00:00,  2.78s/it]


In [4]:
result_df = pd.DataFrame(results).sort_values(by='id')
result_df.to_excel(OUTPUT_EXCEL, index=False)
print(f"处理完成，结果已保存到：{OUTPUT_EXCEL}")

处理完成，结果已保存到：output_file.xlsx
